<a href="https://colab.research.google.com/github/awannabecs/MY-FUNCTION-NAME/blob/main/PROJECT_1_TEXT_CLASSIFICATION_DEPRESSION_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#import the libraries 
import numpy as  np
import pandas as pd 
import tensorflow as tf
import nltk 
import
nltk.download("punkt")
tokenizer = nltk.word_tokenize
tokenizer("go out")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['go', 'out']

In [4]:
#import the dataset
df = pd.read_csv("/content/drive/MyDrive/MAIN_DATA")

In [32]:
#create the word2number dictionary
tokens = []
for i in range(len(df)):
  try:
    tokens.extend(tokenizer(df.iloc[i]["text"].lower()))
  except Exception as e:
    print(df.iloc[i]["text"])

tokens = set(tokens)

nan
nan
nan
nan
nan


In [33]:
tokens

{'x45',
 'ashleylynch',
 '........',
 'breakdown',
 'mariagm350',
 'fantasy',
 'a…',
 'woke',
 '//t.co/ul8mardfhm',
 'might',
 'mother',
 'p',
 'introvert',
 'multi-centre',
 'if',
 'slip',
 'manpower',
 'aww',
 'jwkanyuira1969',
 'yk',
 'milano',
 'astro_lass',
 'gxbby182',
 'so',
 '//t.co/lkxzwxn1fb',
 'aling',
 'unlimited',
 '_kimimi',
 'cci_inter',
 'is.|0|0|0|',
 'wo',
 'boring',
 'what',
 's',
 'lift',
 'newsdistribution',
 'mountain',
 'frtechstartups',
 'mrcfluegel',
 'fat',
 'forward',
 'asia',
 '//t.co/x31iw7dile',
 '⭐',
 'jo',
 'nearly',
 'ca',
 'fake',
 'romanticised',
 'computer',
 'smaller',
 'goodmorning',
 'roofjackson',
 'emotional',
 'gym',
 'adults',
 'benefit',
 'cream',
 'make',
 'upcoming',
 'see',
 'motherfucker',
 'saying',
 'insofar',
 'saddest',
 'oh',
 'afternoon',
 'nz',
 'adamkuczynski',
 'mnwreeen',
 'honey',
 'mrsrobinsonme',
 'grades',
 'wobblycollie',
 'think',
 'watching',
 'somebody',
 'social',
 'many',
 'resignations',
 'kidxsa',
 'z_intl_agency',
 